In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn import model_selection, preprocessing
import xgboost as xgb
import datetime
from sklearn.model_selection import train_test_split

# df_train = pd.read_csv('../input/train.csv', parse_dates=['timestamp'])
# df_test = pd.read_csv('../input/test.csv', parse_dates=['timestamp'])

train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [2]:
train.head(10)

In [3]:
print(train.describe())

In [4]:
train.timestamp.describe()

In [6]:
train.info()

In [7]:
train.sub_area.unique()

In [8]:
train.product_type.unique()

In [9]:
train.groupby('sub_area').sub_area.count()

In [10]:
train.groupby('ecology').sub_area.count()

In [12]:
train.groupby('state').sub_area.count()

In [15]:
Data_stat=train.groupby('state').sub_area.count().sort_values()
Data_stat

In [17]:
type(Data_stat)

In [18]:
train.dtypes

In [19]:
train.price_doc.dtype

In [20]:
print(train.columns)

In [21]:
train=train.fillna(-999)
pd.isnull(train).any()

In [22]:
pd.isnull(test).any()

In [23]:
test=test.fillna(-999)

**Encoding categorical features: 
**assign a numeric value to each categorical feature in our training and test sets. Sklearn's preprocessing unit has a tool called LabelEncoder() which can do just that for us.

In [25]:
y_train = train["price_doc"]
x_train = train.drop(["id", "price_doc","timestamp" ], axis=1)
x_test = test.drop(["id","timestamp"], axis=1)
id_test = test['id']

In [ ]:
# pd.concat([___, ____])
#split

In [26]:
numeric_columns= x_train.columns[x_train.dtypes!='object']
numeric_columns

In [27]:
x_train.dtypes.value_counts()

In [28]:
int64_columns= x_train.columns[x_train.dtypes=='int64']
int64_columns

In [30]:
x_train['sub_area']

In [31]:
for c in x_train.columns:
    if x_train[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_train[c].values)) 
        x_train[c] = lbl.transform(list(x_train[c].values))
        #x_train.drop(c,axis=1,inplace=True)

In [32]:
x_train['sub_area']

In [33]:
for c in x_test.columns:
    if x_test[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_test[c].values)) 
        x_test[c] = lbl.transform(list(x_test[c].values))
        #x_test.drop(c,axis=1,inplace=True) 

In [35]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtrain = xgb.DMatrix(x_train, y_train)
dtest = xgb.DMatrix(x_test)


In [ ]:
# fig, ax = plt.subplots(1, 1, figsize=(8, 13))
# xgb.plot_importance(model, max_num_features=50, height=0.5, ax=ax)

In [ ]:
# cv_output = xgb.cv(xgb_params, dtrain, num_boost_round=1000, early_stopping_rounds=20,
#     verbose_eval=50, show_stdv=False)
# cv_output[['train-rmse-mean', 'test-rmse-mean']].plot()

In [36]:
# num_boost_rounds = len(cv_output)
# print( num_boost_rounds )
num_boost_rounds = 384
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round= num_boost_rounds)

In [39]:
y_predict = model.predict(dtest)
output = pd.DataFrame({'id': id_test, 'price_doc': y_predict})
output.head()


output.to_csv('xgbSb.csv', index=False)


In [ ]:
pd.read_csv('../output/xgbSb.csv')